In [439]:
with open("res/21_1.txt") as f:
    lines = f.read().split("\n")

In [440]:
from collections import defaultdict

to_compute = {}
available = {}
dependant = defaultdict(list)
for line in lines:
    monkey, *rest = line.split()
    monkey = monkey[:-1]
    if len(rest) > 1:
        first, op, second = rest
        dependant[first].append(monkey)
        dependant[second].append(monkey)
        to_compute[monkey] = rest
    else:
        available[monkey] = rest[0]

In [441]:
import operator

operations = {"+": operator.add, "-": operator.sub, "*": operator.mul, "/": operator.floordiv}
reverse_operations = {"-": "+", "+": "-", "/": "*", "*": "/"}
def find_available(available, to_compute, part_two=False):
    available = available.copy()
    to_compute = to_compute.copy()
    while len(to_compute) != 0:
        tmp = {}
        for key in available:
            monkey_list = dependant[key]
            for monkey in monkey_list:
                if monkey in available or monkey in tmp:
                    continue
                first, op, second = to_compute[monkey]
                if (first != key and first in available) or (second != key and second in available):
                    if part_two:
                        tmp[monkey] = available[first] if first != 'humn' else 'humn', available[second] if second != 'humn' else 'humn', op
                    else:
                        tmp[monkey] = available[first], available[second], op
                    del to_compute[monkey]
        available.update(tmp)
    return available

In [442]:
def calc(x, y, sign):
    if x != 'humn':
        x = int(x) if type(x) in (str, int) else calc(*x)
    if y != 'humn':
        y = int(y) if type(y) in (str, int) else calc(*y)
    if type(x) == type(y) == int:
        return operations.get(sign)(x,y)
    return x, y, sign

In [443]:
def reverse_calc(first, second, sign, accumulated):
    if first == 'humn':
        accumulated[0] = (accumulated[0], second, reverse_operations[sign])
        return
    if second == 'humn':
        if sign == '/':
            accumulated[0] = (first, accumulated[0], sign)
        else:
            accumulated[0] = (first, accumulated[0], reverse_operations[sign])
        return
    if type(first) == int:
        if sign in ('/', '-'):
            accumulated[0] = (first, accumulated[0], sign)
        else:
            accumulated[0] = (accumulated[0], first, reverse_operations[sign])
        reverse_calc(*second, accumulated)
        return
    if type(second) == int:
        accumulated[0] = (accumulated[0], second, reverse_operations[sign])
        reverse_calc(*first, accumulated)
        return
    assert 0

In [444]:
bef = find_available(available, to_compute)['root']
calc(*bef)

In [445]:
acc = [0]
bef = find_available(available, to_compute, part_two=True)['root']
reverse_calc(*calc(*bef)[:-1], '-', acc)
calc(*acc[0][0])